In [62]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
!pip install --upgrade mplfinance
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
import mplfinance as mpl 
from datetime import datetime
import glob
from PIL import Image
import cv2
import gc
import gc
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('agg')
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [63]:



def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")


def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols


def download_data(symbols, periodd, intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}", end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,
                           interval=intervall, progress=False, show_errors=False)
        if data.empty:
           pass
        else:
            data.to_csv(f"/content/data/{symbol}.csv")
    except:
       print("Error!")
  print(" ")


def extract_data(how_many_future_candles, how_many_past_candles, each_row_past):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 1
  for file in files:
     print(f"File Number {index}:", end=" ")
     each_file_proc(file, now, how_many_future_candles,
                    how_many_past_candles, each_row_past)
     index = index + 1
  print(" ")
  return now


def each_file_proc(file, now, how_many_future_candles, how_many_past_candles, each_row_past):

    address = f"/content/data/{file}"
    data = pd.read_csv(address)
    data = data.dropna()
    dfs = []
    suggs = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    data = data.astype(float)
    for i in range(each_row_past, data.shape[0]-which_future_or_past):

        rows = data[i-each_row_past:i, :4]

        past_candles = []
        #for z in range(1, how_many_past_candles+1):
        #  past_candles.append(data[i-z][3])
        #past_candles = sum(past_candles)/len(past_candles)
        next_candles = []
        for z in range(0, how_many_future_candles):
          next_candles.append(data[i+z][3])
        next_candles = sum(next_candles)/len(next_candles)
        if next_candles > data[i-1][3]:
          suggs.append(1)
        else:
          suggs.append(0)

        x = pd.DataFrame(rows, columns=["Open", "High", "Low", "Close"])
        del rows
        del past_candles
        del next_candles
        dfs.append(x)
        del x
    del data

    index = 0
    all_n = np.array(dfs).shape[0]
    for df in dfs:
        date_list = []
        rrng = np.array(df).shape[0]
        for i in range(rrng):
            date_list.append(datetime.fromordinal(
                datetime.today().toordinal()+i).strftime('%Y-%m-%d'))
        df.index = date_list
        df.index.name = "Date"
        df.index = pd.to_datetime(df.index)
        right_now = datetime.now().strftime("%H%M%S%f")
        address = f"/content/extracted/{now}/{right_now}_{suggs[index]}.png"
        mpl.plot(df, type="candle", style="yahoo", axisoff=True,
                           scale_padding=0.2, returnfig=True, savefig=address, tight_layout=True,figsize =(3,2),)
        plt.close("all")
        del df
        del date_list
        del rrng
        del i
        del right_now
        gc.collect()



        index = index + 1
        if index % 10 == 0:
            print(f"{index}/{all_n}", end=" ")
        if index % 190 == 0:
          print(" ")

    print("")
    del dfs
    del suggs
    del index
    del all_n
    gc.collect()



def start(how_many_future_candles, how_many_past_candles, each_row_past):
    folder_name = extract_data(
        how_many_future_candles, how_many_past_candles, each_row_past)
    return folder_name


In [66]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
symbols = ["btc-usd","eth-usd","trx-usd","ada-usd","bnb-usd","ltc-usd","xrp-usd"]
download_data(symbols,"1y","1d")

Symbols : 1500
Data Folder Removed
 


In [84]:

how_many_future_candles = 5
how_many_past_candles = 5
each_row_past = 75

global which_future_or_past
which_future_or_past = None
if how_many_future_candles > how_many_past_candles:
    which_future_or_past = how_many_future_candles
else:
    which_future_or_past = how_many_past_candles

folder_name = start(how_many_future_candles,how_many_past_candles,each_row_past)
len(os.listdir(f"/content/extracted/{folder_name}"))

Files In Data : 7
Processing File:
File Number 1: 10/286 20/286 30/286 40/286 50/286 60/286 70/286 80/286 90/286 100/286 110/286 120/286 130/286 140/286 150/286 160/286 170/286 180/286 190/286  
200/286 210/286 220/286 230/286 240/286 250/286 260/286 270/286 280/286 
File Number 2: 10/286 20/286 30/286 40/286 50/286 60/286 70/286 80/286 90/286 100/286 110/286 120/286 130/286 140/286 150/286 160/286 170/286 180/286 190/286  
200/286 210/286 220/286 230/286 240/286 250/286 260/286 270/286 280/286 
File Number 3: 10/286 20/286 30/286 40/286 50/286 60/286 70/286 80/286 90/286 100/286 110/286 120/286 130/286 140/286 150/286 160/286 170/286 180/286 190/286  
200/286 210/286 220/286 230/286 240/286 250/286 260/286 270/286 280/286 
File Number 4: 10/286 20/286 30/286 40/286 50/286 60/286 70/286 80/286 90/286 100/286 110/286 120/286 130/286 140/286 150/286 160/286 170/286 180/286 190/286  
200/286 210/286 220/286 230/286 240/286 250/286 260/286 270/286 280/286 
File Number 5: 10/286 20/286 30/2

2002

In [93]:
#folder_name = '083812'
from PIL import Image
import cv2
label = []
data  = []
files = os.listdir(f"/content/extracted/{folder_name}/")
for i, image_name in enumerate(files):
  if image_name.split(".")[1] == "png":
    image = cv2.imread(f"/content/extracted/{folder_name}"+"/"+image_name,0)
    dim = (128, 128)
    resized = cv2.resize(image, dim)
    data.append(np.array(resized))
    sugg = image_name.split("_")[1].split(".")[0]
    label.append(int(sugg))
np.array(data).shape

(2002, 128, 128)

In [94]:
def scaler(array):
  scaler = MinMaxScaler()
  scaler.fit(array)
  return scaler.transform(array)
data = np.array(data)
new_dd = []
for i in data:
  new_data = []
  for x in i:
    nx = scaler(np.array(x).reshape(-1,1))
    new_data.append(nx)
  new_dd.append(new_data)
np.array(new_dd).shape
data = np.array(new_dd).reshape(-1,data.shape[1],data.shape[2])
new_data = []

In [95]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
x_train , x_test , y_train , y_test = train_test_split(data,label,test_size=0.1)
data = None
label = None
print(f"********** {len(x_train)} {len(x_test)}")
nytrain = []
for i in y_train:
  if i == 1:
    nytrain.append([1,0])
  else:
    nytrain.append([0,1])
nytest = []
for i in y_test:
  if i == 1:
    nytest.append([1,0])
  else:
    nytest.append([0,1])

y_train = nytrain
y_test = nytest
x_train = np.array(x_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
y_train = np.array(y_train)

********** 1801 201


In [ ]:
xx = 0
for i in x_test[0]:
  for x in i:
    if x != 0:
      xx += 1
      print(x)
xx

In [103]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense


model = Sequential()
model.add(Conv2D(50, (4, 4),activation="relu", input_shape=(128, 128,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(100, (4, 4),activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(200, (4, 4),activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(200, (4, 4),activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(750,activation="relu"))
model.add(Dense(750,activation="relu"))
model.add(Dense(750,activation="relu"))
model.add(Dense(750,activation="relu"))
model.add(Dense(2,activation="sigmoid"))

model.compile(loss='categorical_crossentropy',
              optimizer='adamax',
              metrics=['accuracy'])



model.summary()


Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_81 (Conv2D)          (None, 125, 125, 50)      850       
                                                                 
 max_pooling2d_79 (MaxPoolin  (None, 62, 62, 50)       0         
 g2D)                                                            
                                                                 
 conv2d_82 (Conv2D)          (None, 59, 59, 100)       80100     
                                                                 
 max_pooling2d_80 (MaxPoolin  (None, 29, 29, 100)      0         
 g2D)                                                            
                                                                 
 conv2d_83 (Conv2D)          (None, 26, 26, 200)       320200    
                                                                 
 max_pooling2d_81 (MaxPoolin  (None, 13, 13, 200)    

In [ ]:
model.fit(x_train,y_train,batch_size=40,epochs=100,validation_data=(x_test,y_test))


Epoch 1/100
46/46 [==============================] - 3s 54ms/step - loss: 0.0257 - accuracy: 0.9895 - val_loss: 1.3264 - val_accuracy: 0.7313
Epoch 2/100
46/46 [==============================] - 2s 48ms/step - loss: 0.0028 - accuracy: 0.9994 - val_loss: 1.4678 - val_accuracy: 0.7363
Epoch 3/100
46/46 [==============================] - 2s 48ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 1.6274 - val_accuracy: 0.7264
Epoch 4/100
46/46 [==============================] - 2s 49ms/step - loss: 4.9581e-04 - accuracy: 1.0000 - val_loss: 1.7199 - val_accuracy: 0.7214
Epoch 5/100
46/46 [==============================] - 2s 49ms/step - loss: 3.2509e-04 - accuracy: 1.0000 - val_loss: 1.8226 - val_accuracy: 0.7264
Epoch 6/100
46/46 [==============================] - 2s 49ms/step - loss: 1.9718e-04 - accuracy: 1.0000 - val_loss: 1.9489 - val_accuracy: 0.7363
Epoch 7/100
46/46 [==============================] - 2s 49ms/step - loss: 1.1424e-04 - accuracy: 1.0000 - val_loss: 2.0818 - val_accurac